In [1]:
import pandas as pd
from scripts.mock.mock_cleaning import create_demand
import pyomo.environ as pyo

# Data import

In [2]:
nodes = pd.read_pickle("../../data/original/nodes.pkl")
channels = pd.read_pickle("../../data/original/channels.pkl")

## Modeling

Change the logic of CHANNELS set construction by considering the pair of nodes as the edge identifier.
In this way I also have to deal with:
- [X] multiedges, aka multiple edges between two nodes. I can not consider those for simplicity
- [X] directed relationship. I need to understand if flipping the channels - as done before - can be reasonable. I also need to determine if the flipped pair of nodes is considered a valid index by pyomo.

In [7]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])

In [8]:
nodes = create_demand(nodes, 100000)

Transaction of 100000 sats.
Sender: HighlighterSupermarket
Receiver: Odin 17.4 10:00.


In [9]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals, bounds=(0, max(nodes["demand"])))

In [10]:
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)

In [11]:
def objective_function(model: pyo.ConcreteModel):
    return sum(model.a[i] * channels.loc[i, "rate_fee"] for i in model.CHANNELS) + sum(model.x[i] * channels.loc[i, "base_fee"] for i in model.CHANNELS)

model.totalCost = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

### Constraints

#### Number of paths rule

This constraint enforces the symmetrical splitting into different channels of the amount along the path. Specifically, the number of used incoming channels for a node shall be equal to the number of used outgoing channels.

$$
\sum_{(i,n) \in E} x_{i,n} - \sum_{(n,j) \in E} x_{n,j} = 0 \text{ } \forall n \in V
$$


In [12]:
def number_path_rule(model: pyo.ConcreteModel, n):
    outgoing = [model.x[(i, j)] for i, j in channels.index if i == n]
    incoming = [model.x[(i, j)] for i, j in channels.index if j == n]
    return sum(incoming) == sum(outgoing)

model.NumberPathConstraint = pyo.Constraint(model.NODES, rule=number_path_rule, name="Number path constraint")

#### Capacity constraint

$$amount_{i,j} \le capacity_{i,j} \times x_{i,j} \text{ } \forall (i,j) \in E$$

In [13]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.a[(a, b)] <=  channels.loc[(a, b), "capacity"] * model.x[(a, b)]

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

#### Flow balance constraint

$$\sum_{(s,i) \in E} amount_{si} - \sum_{(i,t) \in E} amount_{it} = b_i \text{ } \forall i \in V$$


In [14]:
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    InFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'incoming_channels'])
    OutFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'outgoing_channels'])
    return  OutFlow + nodes.loc[n, "demand"] == InFlow

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True) 

## Solving the model

In [15]:
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, tee=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())


Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpak5ttg0e.pyomo.lp -stat=1 -solve -solu /tmp/tmpak5ttg0e.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 95188 (-18678) rows, 166432 (-13364) columns and 477128 (-61160) elements
Statistics for presolved model
Original problem has 89898 integers (89898 of which binary)
Presolved problem has 83216 integers (83216 of which binary)
==== 38072 zero objective 3879 different
==== absolute objective values 3879 different
==== for integers 31696 zero objective 1018 different
==== for integers absolute objective values 1018 different
===== end objective counts


Problem has 95188 rows, 166432 columns (128360 with objective) and 477128 elements
There are 10564 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 83216 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fix

In [16]:
from decimal import Decimal
DF_channels = pd.DataFrame()
c = 0
for index, value in model.a.extract_values().items():
    if value != 0:
        DF_channels.loc[c, "source"] = index[0]
        DF_channels.loc[c, "destination"] = index[1]
        DF_channels.loc[c, "source-alias"] = nodes.loc[index[0], "alias"]
        DF_channels.loc[c, "destination-alias"] = nodes.loc[index[1], "alias"]
        DF_channels.loc[c, "capacity"] = channels.loc[index, "capacity"]
        DF_channels.loc[c, "amount"] = Decimal(value)
        DF_channels.loc[c, "base_fee"] = channels.loc[(index[0], index[1]), "base_fee"]
        DF_channels.loc[c, "rate_fee"] = channels.loc[(index[0], index[1]), "rate_fee"]
        c += 1

DF_channels_pos = DF_channels[DF_channels["amount"]!=0]
DF_channels_pos

,source,destination,source-alias,destination-alias,capacity,amount,base_fee,rate_fee
0,027fc4bbf73eeb128262948c8ad068ec243e569ead212f...,02cfdc6b60e5931d174a342b20b50d6a2a17c6e4ef8e07...,HighlighterSupermarket,Voltage-C2,500000.0,100000,1.000,0.000240
1,02cfdc6b60e5931d174a342b20b50d6a2a17c6e4ef8e07...,035adfeca8fc95de5e2e9dbf9be39ac577b510e7dad0ad...,Voltage-C2,TennisNbtc,7100000.0,100000,0.000,0.000000
2,030751b547f0e59c40086bd6a2b5efbaaccfa850aa5af5...,02c551a80ba5758c845fc4d554a234e119082fa36f30d1...,RatPoison,Odin 17.4 10:00,1000000.0,100000,1.000,0.000001
3,035adfeca8fc95de5e2e9dbf9be39ac577b510e7dad0ad...,03a93b87bf9f052b8e862d51ebbac4ce5e97b5f4137563...,TennisNbtc,cyberdyne.sh,10000000.0,100000,0.100,0.000000
4,03a93b87bf9f052b8e862d51ebbac4ce5e97b5f4137563...,030751b547f0e59c40086bd6a2b5efbaaccfa850aa5af5...,cyberdyne.sh,RatPoison,4000000.0,100000,0.001,0.000000


In [17]:
DF_channels[DF_channels["amount"]> DF_channels["capacity"]]

,source,destination,source-alias,destination-alias,capacity,amount,base_fee,rate_fee


In [ ]:
DF_fixed = pd.DataFrame()
c = 0
for index, value in model.x.extract_values().items():
    if value != 0:
        DF_fixed.loc[c, "source"] = index[0]
        DF_fixed.loc[c, "destination"] = index[1]
        DF_fixed.loc[c, "used"] = Decimal(value)
        c += 1

DF_fixed_pos = DF_fixed[DF_fixed["used"]!=0]
DF_fixed_pos

In [ ]:
intersection = DF_fixed_pos.merge(DF_channels_pos, on=["source", "destination"], how="outer")
intersection